In [21]:
%load_ext autoreload
%autoreload 2

from cellflow.data import MappedCellData

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from pathlib import Path

data_path = Path("/lustre/groups/ml01/workspace/100mil/tahoe.zarr")

In [25]:
mcd = MappedCellData.read_zarr(data_path)

In [27]:
mcd.src_cell_data

{0: <Array file:///lustre/groups/ml01/workspace/100mil/tahoe.zarr/src_cell_data/0 shape=(60135, 300) dtype=float32>,
 1: <Array file:///lustre/groups/ml01/workspace/100mil/tahoe.zarr/src_cell_data/1 shape=(13778, 300) dtype=float32>,
 10: <Array file:///lustre/groups/ml01/workspace/100mil/tahoe.zarr/src_cell_data/10 shape=(50552, 300) dtype=float32>,
 11: <Array file:///lustre/groups/ml01/workspace/100mil/tahoe.zarr/src_cell_data/11 shape=(39854, 300) dtype=float32>,
 12: <Array file:///lustre/groups/ml01/workspace/100mil/tahoe.zarr/src_cell_data/12 shape=(55202, 300) dtype=float32>,
 13: <Array file:///lustre/groups/ml01/workspace/100mil/tahoe.zarr/src_cell_data/13 shape=(56439, 300) dtype=float32>,
 14: <Array file:///lustre/groups/ml01/workspace/100mil/tahoe.zarr/src_cell_data/14 shape=(36803, 300) dtype=float32>,
 15: <Array file:///lustre/groups/ml01/workspace/100mil/tahoe.zarr/src_cell_data/15 shape=(51949, 300) dtype=float32>,
 16: <Array file:///lustre/groups/ml01/workspace/100

In [28]:
from cellflow.data import ReservoirSampler

In [ ]:
rs = ReservoirSampler(mcd, batch_size=1024, pool_size=3, replacement_prob=0.01)


In [ ]:
import numpy as np
rng = np.random.default_rng(0)
rs.init_pool(rng)


In [40]:
import time
iter_times = []
rng = np.random.default_rng(0)
start_time = time.time()
for iter in range(40):
    batch = rs.sample(rng)
    end_time = time.time()
    iter_times.append(end_time - start_time)
    start_time = end_time

print("average time per iteration: ", np.mean(iter_times))
print("iterations per second: ", 1 / np.mean(iter_times))


KeyError: np.int64(37)

In [3]:
ztd = ZarrTrainingData.read_zarr(data_paths[0])



In [7]:
stats = calculate_memory_cost(ztd, 0)

In [8]:
print(format_memory_stats(stats, summary=True))

Memory statistics for source index 0 with 194 targets:
- Source cells: 60135 cells, 68.82 MB
- Total memory: 548.11 MB

Target memory summary:
- Total: 479.28 MB
- Average: 2.47 MB
- Min: 44.53 KB
- Max: 6.35 MB
- Range: 6.31 MB

Condition memory: 4.55 KB


In [13]:
ztd_stats = {}
for i in range(ztd.n_controls):
    ztd_stats[i] = calculate_memory_cost(ztd, i)


In [16]:
def print_average_memory_per_source(stats_dict):
    """Print the average total memory per source index.
    
    Parameters
    ----------
    stats_dict
        Optional pre-calculated memory statistics dictionary.
        If None, statistics will be calculated for all source indices.
    """
    
    
    # Extract total memory for each source index
    total_memories = [stats["total_memory"] for stats in stats_dict.values()]
    
    # Calculate statistics
    avg_memory = np.mean(total_memories)
    min_memory = np.min(total_memories)
    max_memory = np.max(total_memories)
    median_memory = np.median(total_memories)
    
    # Format the output
    def format_bytes(bytes_value):
        for unit in ["B", "KB", "MB", "GB"]:
            if bytes_value < 1024 or unit == "GB":
                break
            bytes_value /= 1024
        return f"{bytes_value:.2f} {unit}"
    
    print(f"Memory statistics across {len(stats_dict)} source indices:")
    print(f"- Average total memory per source: {format_bytes(avg_memory)}")
    print(f"- Minimum total memory: {format_bytes(min_memory)}")
    print(f"- Maximum total memory: {format_bytes(max_memory)}")
    print(f"- Median total memory: {format_bytes(median_memory)}")
    print(f"- Range: {format_bytes(max_memory - min_memory)}")
    
    # Identify source indices with min and max memory
    min_idx = min(stats_dict.keys(), key=lambda k: stats_dict[k]["total_memory"])
    max_idx = max(stats_dict.keys(), key=lambda k: stats_dict[k]["total_memory"])
    
    print(f"\nSource index with minimum memory: {min_idx} ({format_bytes(min_memory)})")
    print(f"Source index with maximum memory: {max_idx} ({format_bytes(max_memory)})")

In [59]:
print_average_memory_per_source(ztd_stats)

Memory statistics across 50 source indices:
- Average total memory per source: 423.18 MB
- Minimum total memory: 4.33 MB
- Maximum total memory: 1.29 GB
- Median total memory: 404.51 MB
- Range: 1.28 GB

Source index with minimum memory: 39 (4.33 MB)
Source index with maximum memory: 22 (1.29 GB)


In [11]:
from cellflow.data import TrainSamplerWithPool
import numpy as np
rng = np.random.default_rng(0)

In [13]:
tswp = TrainSamplerWithPool(ztd, batch_size=1024, pool_size=20, replacement_prob=0.01)
tswp.init_pool_n_cache(rng)

Computing source to cell data idcs: 100%|██████████| 50/50 [00:00<00:00, 1232.06it/s]


In [ ]:
import time
iter_times = []
rng = np.random.default_rng(0)
start_time = time.time()
for iter in range(40):
    batch = tswp.sample(rng)
    end_time = time.time()
    iter_times.append(end_time - start_time)
    start_time = end_time

print("average time per iteration: ", np.mean(iter_times))
print("iterations per second: ", 1 / np.mean(iter_times))


replaced 47 with 34
replaced 32 with 30


In [64]:
tswp.get_pool_stats()

{'pool_size': 20,
 'avg_usage': 1.95,
 'unique_sources': 20,
 'pool_elements': array([31, 18, 47, 34, 12, 35, 29, 23, 32, 14,  6, 41, 25,  3,  1, 49, 24,
        10, 46, 33]),
 'usage_counts': array([2, 2, 3, 2, 1, 0, 2, 2, 2, 0, 3, 1, 2, 0, 3, 3, 2, 6, 1, 2])}